In [ ]:
import os
import numpy as np
import scipy
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import IPython.display as ipd
import librosa
import soundfile as sf

import librosa
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


def process_audio_data(queen_file, no_queen_file):
    def load_and_process_data(training_file, label):
        training_data, sampling_rate = librosa.load(training_file, sr=None)

        # Create segments for training data
        segment_length = 3  # seconds
        segment_length_frames = int(segment_length * sampling_rate)
        training_data_segments = [
            training_data[i:i + segment_length_frames]
            for i in range(0, len(training_data), segment_length_frames)
        ]

        # Extract MFCC features and assign labels
        mfccs = []
        labels = []

        for segment in training_data_segments:
            mfcc = librosa.feature.mfcc(y=segment, sr=sampling_rate, n_mfcc=13)
            mfccs.append(mfcc)
            labels.append(label)

        # Calculate the minimum length among all segments
        min_length = min(mfcc.shape[1] for mfcc in mfccs)

        # Trim MFCC matrices to the minimum length
        mfccs = [mfcc[:, :min_length] for mfcc in mfccs]

        return mfccs, labels

    # Load and process data for queen class
    queen_mfccs, queen_labels = load_and_process_data(queen_file, label=1)

    # Load and process data for no queen class
    no_queen_mfccs, no_queen_labels = load_and_process_data(no_queen_file, label=0)

    # Combine the data from "queen" and "no queen" segments and labels:
    X = np.vstack((queen_mfccs, no_queen_mfccs))
    y = np.hstack((queen_labels, no_queen_labels))

    # Reshape X to have two dimensions
    X = X.reshape(X.shape[0], -1)  # Flatten the last two dimensions

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create an SVM classifier (you can choose different kernels and parameters)
    clf = svm.SVC(kernel='linear')

    # Train the classifier on the training data
    clf.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = clf.predict(X_test)

    # Calculate accuracy and print the classification report
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy of MFCC Features: {accuracy * 100:.2f}%")

    report = classification_report(y_test, y_pred)
    print("\nClassification Report:\n", report)

# File paths
queen_training_file = "QueenBee_Training_45mins.wav"
no_queen_training_file = "No_QueenBee_Training_45mins.wav"

# Process audio data using the encapsulated function
process_audio_data(queen_training_file, no_queen_training_file)
